# Instalations

In [1]:
pip install pymysql sqlalchemy-redshift pykeepass

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pykeepass import PyKeePass
import logging
import getpass
import os
from datetime import datetime
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
import boto3 #Save in S3
from pathlib import Path

In [3]:
S3_bucket_name = 'edfred-edfre-sbx-eu-west-1-solar-radiation-data'
S3_CSV_FOLD = r'EtudeWindIndex/Real'

# Accès aux données Keepass

In [4]:
#Get logging information stored in keepass
Configuration_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Windga/WindGa_hourly/Configuration')
KEEPASS_FILE = Configuration_path/'RDL.kdbx'

logging.basicConfig(format=' %(asctime)s -  %(levelname)s -  %(message)s', handlers = [logging.StreamHandler()])
logging.getLogger().setLevel(logging.INFO)

print('Provide password to get acces to Keepass file:')
password = getpass.getpass()

try:  
    kp = PyKeePass(KEEPASS_FILE, password=password)
    logging.info('Keepass data loaded')
except:
    logging.exception('Cannot open the keepass file:')

Provide password to get acces to Keepass file:


 ·········


 2022-02-01 13:55:37,542 -  INFO -  Keepass data loaded


# Connection à la base de données

In [5]:
def connexion_setup(kp_con_name='SBX-RDS'):

    credential = kp.find_entries(title=kp_con_name, first=True)

    con = credential.url.split('//')[0] + '//' + \
    credential.username +':'+ \
    credential.password + \
    credential.url.split('//')[1]
    logging.debug(f'connexion string: {con}')

    con_engine = sqlalchemy.create_engine(con)

    return con_engine

In [6]:
#Read mapping file
VAR_MAPPING_FILE = Configuration_path/'10min_variables_iec_to_std_names_mapping.csv'
df = pd.read_csv(VAR_MAPPING_FILE)
df.head()

#Mapping dict
iec2std = df.loc[:,['iec_attribute61400','business_description']].set_index('iec_attribute61400')['business_description'].to_dict()
std2iec = df.loc[:,['iec_attribute61400','business_description']].set_index('business_description')['iec_attribute61400'].to_dict()

#Récupération des données utiles
mask_features = [False]
List_features = ['active_power_avg','wind_speed_avg']
for feature in List_features :
    mask_features = mask_features | (df.business_description == feature)
temperature_features = df.loc[mask_features,:]
features_selection_str = str(list(temperature_features.iec_attribute61400)).replace("'", '').replace("[", "").replace("]", "")

# Extraction de données

In [7]:
#Liste des projets à extraire
start_project = 'PEZI'
start_month = '2020-01'
end_month = '2021-11'

In [8]:
months_range = pd.date_range(start=start_month, end=end_month, freq='MS')
months_list = [str(months_range[i])[0:7] for i in range(len(months_range))]

#On récupère la liste des projets
List_projects = pd.read_csv(Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/list_projects.csv'),sep=';').project.tolist()
List_projects = List_projects[List_projects.index(start_project):]
#List_projects = ['ESPS']

In [ ]:
for PROJECT in List_projects :
    
    logging.info("project : {}".format(PROJECT))
    
    _10min_project = pd.DataFrame(columns=['asset_id','ts'] + List_features)
    
    #Number of WTGS in this project from WDM
    query = f'''SELECT project_code, iec_eqpt_code, wt_neighbor_01, power_curve_code
    FROM wdm.adm_eqpt_wt 
    WHERE project_code = '{PROJECT}' ;
    '''
    
    with connexion_setup('SBX-WDM').connect() as conn:
        wtg_eqpt_codes = pd.read_sql_query(sql=query, con=conn, params={})
    
    WTGS = tuple(wtg_eqpt_codes.iec_eqpt_code)
        
    #10 minutes data extraction
    for MONTH in months_list :
        
        logging.info("month : {}".format(MONTH))
        
        FROM_DATE =  MONTH + '-01 00:00:00'  # included
        if int(MONTH[-2:])==12:
            TO_DATE = f'{int(MONTH[:-3]) + 1}-01-01 00:00:00'      # not included 
        elif int(MONTH[-2:])>=9:
                TO_DATE = f'{MONTH[:-3]}-{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
        else :
            TO_DATE = f'{MONTH[:-3]}-0{int(MONTH[-2:]) + 1}-01 00:00:00'      # not included 
            TO_DATE = f'2021-12-01 00:00:00'      # not included
            
        DATES = [d.strftime('%Y%m%d') for d in pd.date_range(FROM_DATE, TO_DATE)]
        _10min_tables = [f'eu_data.tur_10m_{date}_q' for date in DATES[:-1]]
        query = ""
        
        for i, table in enumerate(_10min_tables):
            if i != 0:
                query = query + "UNION\n"
            query = query + f'''SELECT  asset_id, 
            ts,
            {features_selection_str}
            FROM {table} WHERE project = '{PROJECT}' AND tech_source = 'PIOEM' \n'''
            
        query = query + "ORDER BY ts ;"
        
        with (connexion_setup('PWU-RSH').connect()) as conn:
            _10min = pd.read_sql_query(sql=query, con=conn)    # query_red(conn, query)
            
        _10min.rename(columns=iec2std, inplace=True)
        
        # re-order columns
        col_selection = _10min.columns.sort_values()
        _10min = _10min.loc[:,col_selection]
        
        _10min_project = pd.concat([_10min_project, _10min])
        
    #On converti le timestamp
    _10min_project['year'] = _10min_project.ts.map(lambda date: date.year)
    _10min_project['month'] = _10min_project.ts.map(lambda date: date.month)
    _10min_project['day'] = _10min_project.ts.map(lambda date: date.day)
    _10min_project['hour'] = _10min_project.ts.map(lambda date: date.hour)
    _10min_project['minute'] = _10min_project.ts.map(lambda date: date.minute)
    
    #On récupère le nom de projet et le numéro de turbine
    _10min_project['project'] = _10min_project.asset_id.map(lambda name: name[0:4])
    _10min_project['turbine'] = _10min_project.asset_id.map(lambda name: int(name[-3:]))
    
    #On garde les informations utiles
    _10min_project.drop(['asset_id','ts'], axis=1, inplace=True)
    #On modifie l'index pour retrouver plus facilement les données
    _10min_project.set_index(['project','turbine','year','month','day','hour','minute'], inplace=True)

    #Sauvegarde sur le notebook
    _10min_path = Path('/home/ec2-user/SageMaker/EtudeWindIndex/Data/Windga/WindGa_hourly/Clean/10min_'+PROJECT+'.csv')
    _10min_project.to_csv(_10min_path, index=False, sep=';')
    
    #Sauvegarde sur le S3
    outfile = '10min_'+PROJECT+'.csv'
    _10min_project.to_csv(f's3://{S3_bucket_name}/{S3_CSV_FOLD}/{outfile}', index=True, sep=';')

 2022-02-01 13:55:37,676 -  INFO -  project : PEZI
 2022-02-01 13:55:37,813 -  INFO -  month : 2020-01
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
 2022-02-01 13:58:35,579 -  INFO -  month : 2020-02
 2022-02-01 14:01:32,961 -  INFO -  month : 2020-03
 2022-02-01 14:04:22,070 -  INFO -  month : 2020-04
 2022-02-01 14:06:54,922 -  INFO -  month : 2020-05
 2022-02-01 14:09:35,721 -  INFO -  month : 2020-06
 2022-02-01 14:11:51,921 -  INFO -  month : 2020-07
 2022-02-01 14:13:55,415 -  INFO -  month : 2020-08
 2022-02-01 14:15:51,636 -  INFO -  month : 2020-09
 2022-02-01 14:16:02,163 -  INFO -  month : 2020-10
 2022-02-01 14:16:11,489 -  INFO -  month : 2020-11
 2022-02-01 14:16:21,248 -